In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# --- Settings ---
model_path = "Crack.pt"          # Trained YOLOv8-Segmentation model
image_path = "c1.jpeg"           # Input image
output_path = "output/img1.jpg"  # Save output image

# Load model
model = YOLO(model_path)

# Load image
img = cv2.imread(image_path)
height, width = img.shape[:2]

# Run inference
results = model(img)[0]

# --- Draw ONLY Segmentation Mask (No Boxes) ---
if results.masks is not None:
    for mask in results.masks.data:
        mask = mask.cpu().numpy()
        mask = cv2.resize(mask, (width, height))
        mask = (mask > 0.5).astype(np.uint8)  # Binary mask (0,1)

        # Create blank color mask - only green channel for cracks
        crack_mask = np.zeros_like(img)
        crack_mask[:, :, 1] = mask * 255  # Green channel

        # Overlay segmentation mask on original image
        img = cv2.addWeighted(img, 1, crack_mask, 0.5, 0)
else:
    print("No segmentation masks detected.")

# --- Show Result ---
cv2.imshow("Crack Segmentation", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# --- Save Output ---
cv2.imwrite(output_path, img)
print(f"Segmented result saved to {output_path}")


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial import distance

# --- Settings ---
model_path = "Crack.pt"      # YOLOv8 segmentation model
image_path = "c1.jpeg"       # Input image
PIXELS_PER_CM = 50           # ✅ 1 cm = 50 pixels (your scale)

# Load YOLO model and image
model = YOLO(model_path)
img = cv2.imread(image_path)
height, width = img.shape[:2]

# Run segmentation
results = model(img)[0]

if results.masks is None:
    print("No cracks detected.")
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    masks = results.masks.data.cpu().numpy()
    crack_centers = []

    for mask in masks:
        mask = cv2.resize(mask, (width, height))
        mask = (mask > 0.5).astype(np.uint8)

        # ✅ Green crack segmentation mask (filled)
        green_mask = np.zeros_like(img)
        green_mask[:, :, 1] = mask * 255  # Green channel only
        img = cv2.addWeighted(img, 1, green_mask, 0.5, 0)

        # Find crack center
        ys, xs = np.where(mask == 1)
        if len(xs) > 0 and len(ys) > 0:
            crack_centers.append((int(np.mean(xs)), int(np.mean(ys))))

    # ✅ If multiple cracks -> calculate gap, else just show segments
    if len(crack_centers) > 1:
        for i, center in enumerate(crack_centers):
            min_dist_px = float("inf")
            nearest = None

            for j, other in enumerate(crack_centers):
                if i != j:
                    dist = distance.euclidean(center, other)
                    if dist < min_dist_px:
                        min_dist_px = dist
                        nearest = other

            # ✅ Convert pixel distance -> centimeters
            gap_cm = min_dist_px / PIXELS_PER_CM

            # ✅ Find midpoint to place text
            mid_x = int((center[0] + nearest[0]) / 2)
            mid_y = int((center[1] + nearest[1]) / 2)

            # ✅ Only show gap in cm (black text)
            cv2.putText(img, f"{gap_cm:.2f} cm", (mid_x, mid_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)  # Black text
            
    #cv2.namedWindow("Crack Segmentation with Gap (cm)", cv2.WINDOW_NORMAL)  # Make window resizable
    #cv2.resizeWindow("Crack Segmentation with Gap (cm)", 800, 600)   

    # ✅ Show final result (no saving)
    cv2.imshow("Crack Segmentation with Gap (cm)", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
